<a href="https://colab.research.google.com/github/christopher-reed/ag_data_project/blob/master/temperature_GDD_KDD_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from ftplib import FTP
from calendar import isleap

In [0]:
#READ ME for .dly files: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
wids = [11,4,2,4]+[5,1,1,1]*31
nam = ['VAL','MFL','QFL','SFL']*31
num = list(range(1,32))*4
num.sort()
val_flg = [nam[i]+str(num[i]) for i in range(len(num))]

#weather station in Aledo County, IL
stat = pd.read_fwf('USC00110072.dly',widths=wids,header=None,index_col=False,names=['ID','YR','MO','VAR']+val_flg)

In [0]:
def var_ext(var,stat):
  sv = pd.Series()
  
  # index of variable to extract
  idx = stat[stat.loc[:,'VAR']==var].index.tolist()
  
  # days in each month incl. leap year day
  days = [31,28,31,30,31,30,31,31,30,31,30,31]
  ldays = [31,29,31,30,31,30,31,31,30,31,30,31]
  
  # convenience variables for years and months
  yr = stat.loc[idx,'YR']
  mo = stat.loc[idx,'MO']
  
  # loop through every instance of the variable to extract
  for i in idx:
    # number of days in month
    if isleap(yr[i]):
      d = ldays[mo[i]-1]
    else:
      d = days[mo[i]-1]
    # only pull values that correspond to days in the month
    out = stat.loc[i][list(range(4,4*(d+1),4))] 
    
    # replace missing values with NaN
    out[out==-9999] = np.nan
    
    # reindex with proper time stamp
    tm = pd.to_datetime({'year':yr[i],'month':mo[i],'day':range(1,d+1)})
    out = pd.Series(out.values,index=tm)*0.1 # convert to measured units
    sv = pd.concat([sv,out])
    
  # get full date vector and expand NaNs for missing data
  date1 = pd.to_datetime({'year':yr[idx[0]],'month':mo[idx[0]],'day':[1]})
  date2 = pd.to_datetime({'year':yr[idx[-1]],'month':mo[idx[-1]],'day':[d]})
  all_dates = pd.date_range(start=date1[0],end=date2[0])
  sv = sv.reindex(index=all_dates)
  return sv

In [56]:
#extract TMAX
tmax = var_ext('TMAX', stat)
tmax.head()

1901-01-01   -6.1
1901-01-02   -2.8
1901-01-03    1.1
1901-01-04      5
1901-01-05    0.6
Freq: D, dtype: object

In [57]:
#extract TMIN
tmin = var_ext('TMIN', stat)
tmin.head()

1901-01-01     -20
1901-01-02   -16.1
1901-01-03   -14.4
1901-01-04    -2.8
1901-01-05      -5
Freq: D, dtype: object

In [0]:
#combine TMAX and TMIN
temp = pd.concat([tmax, tmin], axis=1)
temp.rename(columns = {0:'TMAX_obs', 1:'TMIN_obs'}, inplace = True)

In [110]:
#make copies of observed max and min temp that we can edit while keeping original intact

temp['TMAX_bound'] = temp['TMAX_obs']
temp['TMIN_bound'] = temp['TMIN_obs']

temp.head(3)

,TMAX_obs,TMIN_obs,TMAX_bound,TMIN_bound
1901-01-01,-6.1,-20,-6.1,-20
1901-01-02,-2.8,-16.1,-2.8,-16.1
1901-01-03,1.1,-14.4,1.1,-14.4


In [0]:
#want to find all observed TMAX above 29 and replace them with 29
temp.loc[temp.TMAX_bound >= 29, 'TMAX_bound'] = 29

In [0]:
#want to find all observed TMAX below 9 and replace them with 9
temp.loc[temp.TMAX_bound <= 9, 'TMAX_bound'] = 9

In [0]:
#want to find all observed TMIN above 29 and replace them with 29
temp.loc[temp.TMIN_bound >= 29, 'TMIN_bound'] = 29

In [0]:
#want to find all observed TMIN below 9 and replace them with 9
temp.loc[temp.TMIN_bound <= 9, 'TMIN_bound'] = 9

In [121]:
#inspect a random middle section of the data
temp.iloc[475:525]

,TMAX_obs,TMIN_obs,TMAX_bound,TMIN_bound
1902-04-21,31.1,12.8,29,12.8
1902-04-22,26.1,11.7,26.1,11.7
1902-04-23,15,0.6,15,9
1902-04-24,23.3,-0.6,23.3,9
1902-04-25,22.8,10,22.8,10
1902-04-26,22.2,3.9,22.2,9
1902-04-27,18.3,2.8,18.3,9
1902-04-28,23.3,-0.6,23.3,9
1902-04-29,21.7,11.1,21.7,11.1
1902-04-30,22.2,7.8,22.2,9


In [0]:
#calculate GDD and add as a column
temp['GDD'] = ((temp['TMAX_bound'] + temp['TMIN_bound'])/2)-9

In [123]:
temp.head()

,TMAX_obs,TMIN_obs,TMAX_bound,TMIN_bound,GDD
1901-01-01,-6.1,-20,9,9,0
1901-01-02,-2.8,-16.1,9,9,0
1901-01-03,1.1,-14.4,9,9,0
1901-01-04,5,-2.8,9,9,0
1901-01-05,0.6,-5,9,9,0


In [133]:
#sanity check to see that not all the GDD values are 0

zeroGDD = temp['GDD'] == 0
nonzeroGDD = temp['GDD'] != 'UCLA Bruins'

sum(zeroGDD)/sum(nonzeroGDD)

0.31794800500486586

In [0]:
#calculate KDD. np.where(condition, value if True, value if False).
temp['KDD'] = np.where(temp['TMAX_obs'] > 29, temp['TMAX_obs'] - 29, 0)

In [137]:
#inspect middle section of data
temp.iloc[475:525]

,TMAX_obs,TMIN_obs,TMAX_bound,TMIN_bound,GDD,KDD
1902-04-21,31.1,12.8,29,12.8,11.9,2.1
1902-04-22,26.1,11.7,26.1,11.7,9.9,0
1902-04-23,15,0.6,15,9,3,0
1902-04-24,23.3,-0.6,23.3,9,7.15,0
1902-04-25,22.8,10,22.8,10,7.4,0
1902-04-26,22.2,3.9,22.2,9,6.6,0
1902-04-27,18.3,2.8,18.3,9,4.65,0
1902-04-28,23.3,-0.6,23.3,9,7.15,0
1902-04-29,21.7,11.1,21.7,11.1,7.4,0
1902-04-30,22.2,7.8,22.2,9,6.6,0


In [175]:
#locate the missing data. we see 960 rows are missing one or more data points.
missingdata = temp[temp.isnull().any(axis=1)]
len(missingdata.index)

960

In [187]:
#remove some missing data by narrowing down to study years
temp = temp.loc['1981-01-01':'2017-12-31']
missingdata = temp[temp.isnull().any(axis=1)]
len(missingdata.index)

506

In [188]:
#what percentage of our data is missing?
len(missingdata.index)/len(temp.index)

0.037442652064525676

In [181]:
#missing data ranges from single points to entire weeks or multiple weeks of missing data
missingdata['1983']

,TMAX_obs,TMIN_obs,TMAX_bound,TMIN_bound,GDD,KDD
1983-07-01,33.3,NaN,29,NaN,NaN,4.3
1983-07-02,32.2,NaN,29,NaN,NaN,3.2
1983-07-03,33.9,NaN,29,NaN,NaN,4.9
1983-07-04,32.2,NaN,29,NaN,NaN,3.2
1983-07-05,25,NaN,25,NaN,NaN,0
1983-07-06,23.9,NaN,23.9,NaN,NaN,0
1983-07-07,28.3,NaN,28.3,NaN,NaN,0
1983-07-08,30,NaN,29,NaN,NaN,1
1983-07-09,31.7,NaN,29,NaN,NaN,2.7
1983-07-10,32.2,NaN,29,NaN,NaN,3.2
